In [1]:
# install the libraries necessary for data wrangling, prediction and result analysis
import json
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, f1_score,precision_score, recall_score
import torch
from numba import cuda
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [ ]:
!pip install -q transformers
!pip install -q simpletransformers
from simpletransformers.classification import ClassificationModel

In [ ]:
import wandb
wandb.login()

In [ ]:
# Clean the GPU cache

cuda.select_device(0)
cuda.close()
cuda.select_device(0)
torch.cuda.empty_cache()

In [ ]:
train_df = pd.read_csv('')
print(train_df)

test_df = train_df[900:1000]
test_df = test_df.drop(['label'],axis = 1)
print(test_df)

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
# Create a list of labels
LABELS = train_df.label.unique().tolist()
LABELS
train_df['label']=train_df['label'].replace(['Other', 'Information/Explanation', 'News', 'Instruction', 'Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Legal', 'Promotion'],[0,1,2,3,4,5,6,7,8])
dev_df = train_df[:100]
print(dev_df)

In [ ]:
# Initialize Wandb
wandb.init(project="Hindi-hyperparameter-search", name="training-and-saving-the-model")

In [ ]:
# Calculate how many steps will each epoch have
# Num steps in epoch = training samples / batch size
steps_per_epoch = int(579/8)
steps_per_epoch

In [ ]:
# Create a TransformerModel and evaluate during training
epoch = 90

roberta_base_model = ClassificationModel(
        "xlmroberta", "TajaKuzman/xlm-roberta-base-multilingual-text-genres",
        use_cuda=True,
        args= {
            "overwrite_output_dir": True,
            "num_train_epochs": epoch,
            "train_batch_size":8,
            "learning_rate": 1e-5,
            # Use these parameters if you want to evaluate during training
            "evaluate_during_training": True,
            "evaluate_during_training_steps": steps_per_epoch*10,
            "evaluate_during_training_verbose": True,
            "use_cached_eval_features": True,
            'reprocess_input_data': True,
            # "labels_list": LABELS,
            # The following parameters (no_cache, no_save) are commented out if I want to save the model
            "no_cache": True,
            # Disable no_save: True if you want to save the model
            "no_save": True,
            "max_seq_length": 512,
            "save_steps": -1,
            # Only the trained model will be saved - to prevent filling all of the space
            "save_model_every_epoch":False,
            "wandb_project": 'Hindi-hyperparameter-search',
            "silent": True,
            }
        )

# Train the model and evaluate during training
roberta_base_model.train_model(train_df, eval_df = dev_df)